## Pré processamento

In [ ]:
import pandas as pd
import os
from joblib import Parallel, delayed
import multiprocessing

In [ ]:
nome_atributos = ['TIPO', 'DATA', 'CODBDI', 'CODNEG', 'TPMERC', 'NOMRES', 'ESPECI', 'PRAZOT', 
                'MODREF', 'PREABE', 'PREMAX', 'PREMIN', 'PREMED', 'PREULT', 'PREOFC', 'PREOFV', 
                'TOTNEG', 'QUATOT', 'VOLTOT', 'PREEXE', 'INDOPC', 'DATVEN', 'FATCOT', 'PTOEXE', 'CODISI', 'DISMES']
posicao_atributos = [(0,2),(2,10),(10,12),(12,24),(24,27),(27,39),(39,49),(49,52),(52,56),
           (56,69),(69,82),(82,95),(95,108),(108,121),(121,134),(134,147),(147,152),
           (152,170),(170,188),(188,201),(201,202),(202,210),(210,217),(217,230),(230,242),(242,245)]

In [ ]:
def converterArquivoBruto(arquivo):

    dados = pd.read_fwf(arquivo, colspecs=posicao_atributos)

    dados.columns = nome_atributos
    
    # removendo ultima linha do tipo 99, esta última linha apenas representa que o arquivo chegou ao fim.
    dados = dados.iloc[:-1]
    
    return dados

In [ ]:
arquivos = os.listdir("dados/")

arquivos

In [ ]:
dados = pd.DataFrame(columns=nome_atributos)

In [ ]:
for arquivo in arquivos:
    dados = dados.append(converterArquivoBruto('dados/'+arquivo))

len(dados)

In [ ]:
dados.tail()

In [ ]:
anos = dados.DATA.str[:4].unique()
anos.sort() # ordenar
anos

In [ ]:
t = dados.loc[dados['DATA'].str[:4] == '2018', 'DATA'].unique()
t[-5:]

### Separar os dados de validação, que são após 24/10/2018

In [ ]:
validacao = dados.loc[dados['DATA']>'20181024']
validacao.head()

In [ ]:
validacao = validacao[['DATA', 'CODNEG', 'NOMRES', 'PREABE', 'PREULT']]
validacao.head()

In [ ]:
# Criar variável classe
validacao['STATUS_POSITIVO'] = 1

# Se o valor de fechamento for menor que o valor de abertura, entao a ação teve diminuição/queda
validacao.loc[validacao['PREULT']<validacao['PREABE'],'STATUS_POSITIVO'] = 0

In [ ]:
validacao['STATUS_POSITIVO'].unique()

In [ ]:
validacao.loc[validacao['CODNEG']=='PETR4'].head()

In [ ]:
validacao.loc[validacao['CODNEG']=='VALE3'].head()

In [ ]:
validacao = validacao[['DATA', 'CODNEG', 'STATUS_POSITIVO']]
validacao.head()

In [ ]:
validacao['DATA'] = validacao['DATA'].astype('int')
validacao.head()

In [ ]:
validacao.loc[validacao['CODNEG'] == 'PETR4T']

In [ ]:
dimensao_data = pd.read_csv('../dados/dimensao_data_tratado.csv', sep=';')
dimensao_data.tail()

In [ ]:
completo = pd.merge(validacao, dimensao_data, on='DATA')
completo.head()

In [ ]:
completo.isnull().values.any()

In [ ]:
len(completo.CODNEG.unique())

In [ ]:
acoes_verificadas = pd.read_csv('../dados/acoes_verificadas.csv')
acoes_verificadas.head()

In [ ]:
completo = completo.loc[completo['CODNEG'].isin(acoes_verificadas.CODNEG.unique())]

In [ ]:
len(completo.CODNEG.unique())

In [ ]:
completo.head()

In [ ]:
# Coletando as ações
acoes = completo.CODNEG.unique()

In [ ]:
def verificarAcao(acao):
    classes = completo.loc[completo['CODNEG'] == acao, 'STATUS_POSITIVO'].unique()

    if (len(classes) == 1): # Se a ação atender a algum desses critérios ela será removida
        print(acao)
        return acao        

In [ ]:
remover_acoes = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(verificarAcao)(acao) for acao in acoes)

In [ ]:
remover_acoes = list(filter(None, remover_acoes))

In [ ]:
len(remover_acoes)

In [ ]:
completo = completo.loc[~completo['CODNEG'].isin(remover_acoes)]

completo.head()

In [ ]:
len(completo.CODNEG.unique())

In [ ]:
completo.to_csv('dados_validacao.csv', index=False, sep=';')